In [1]:
# !pip install mtcnn
# !pip install tensorflow
# !pip install facenet-pytorch

Plotting densenet-vit logs

In [17]:
import os
import matplotlib.pyplot as plt

# Function to extract metrics from a log file for a given epoch
def extract_metrics(file_lines, epoch):
    metrics = {
        "Training BCE supervision Loss": float(file_lines[epoch + 1].split()[-1]),
        "Training BCE classification Loss": float(file_lines[epoch + 2].split()[-1]),
        "Validation BCE supervision Loss": float(file_lines[epoch + 3].split()[-1]),
        "Validation BCE classification Loss": float(file_lines[epoch + 4].split()[-1]),
        "Validation Accuracy": float(file_lines[epoch + 5].split()[-1]),
    }
    return metrics

# Function to iterate through log files, extract metrics, and save them in a text file
def process_log_files(folder_path, output_file):
    # Check if the output file exists, and create it if not
    with open(output_file, "a") as output:
        if output.tell() == 0:  # Check if the file is empty
            output.write("Metrics Summary:\n\n")

    with open(output_file, "w") as output:  # Use "a" mode for append
        filenames = sorted(os.listdir(folder_path))  # Ensure files are sorted by name
        for filename in filenames:
            if filename.endswith(".log"):
                with open(os.path.join(folder_path, filename), "r") as file:
                    file_lines = file.readlines()

                    # Find the line containing "Epoch" to extract the epoch number
                    for i, line in enumerate(file_lines):
                        if "Epoch" in line:
                            # Extracting the epoch number
                            epoch_index = line.find("[")
                            if epoch_index != -1:
                                epoch = int(line[epoch_index + 1 : epoch_index + 2])
                                output.write(f"epoch {epoch}:\n")

                                # Extract metrics for the current epoch
                                metrics = extract_metrics(file_lines, i)
                                for key, value in metrics.items():
                                    output.write(f"{key}: {value}\n")

                                output.write("\n")
                            else:
                                print(f"Warning: Skipping line without 'Epoch' format in {filename}, line {i + 1}")

# Function to read metrics from the text file
def read_metrics_from_file(file_path):
    metrics_data = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        current_epoch = None
        current_metrics = {}

        for line in lines:
            line = line.strip()
            if line.startswith('epoch'):
                # Save previous epoch's metrics
                if current_epoch is not None:
                    metrics_data.append(current_metrics)
                # Start a new epoch
                current_epoch = int(line.split()[1][:-1])
                current_metrics = {'epoch': current_epoch}
            elif line:
                key, value = line.split(': ')
                current_metrics[key] = float(value)

    # Add the last epoch's metrics
    if current_epoch is not None:
        metrics_data.append(current_metrics)

    return metrics_data

# Function to plot graphs from metrics data
def plot_graphs_from_metrics(metrics_data, folder_name):
    epochs = [metrics['epoch'] for metrics in metrics_data]

    train_supervision_losses = [metrics['Training BCE supervision Loss'] for metrics in metrics_data]
    val_supervision_losses = [metrics['Validation BCE supervision Loss'] for metrics in metrics_data]

    train_classification_losses = [metrics['Training BCE classification Loss'] for metrics in metrics_data]
    val_classification_losses = [metrics['Validation BCE classification Loss'] for metrics in metrics_data]

    val_accuracies = [metrics['Validation Accuracy'] for metrics in metrics_data]

    # Plotting graphs
    plt.figure(figsize=(12, 5))

    # Plotting supervision losses
    plt.subplot(1, 3, 1)
    plt.plot(epochs, train_supervision_losses, label='Train')
    plt.plot(epochs, val_supervision_losses, label='Validation')
    plt.title('Supervision Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plotting classification losses
    plt.subplot(1, 3, 2)
    plt.plot(epochs, train_classification_losses, label='Train')
    plt.plot(epochs, val_classification_losses, label='Validation')
    plt.title('Classification Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plotting validation accuracy
    plt.subplot(1, 3, 3)
    plt.plot(epochs, val_accuracies, label='Validation Accuracy')
    plt.title('Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()

    # Save the plot with the folder name
    plot_file_name = f"output/plot_{folder_name}.png"
    plt.savefig(plot_file_name)
    plt.close()  # Close the plot to avoid displaying it

# Set the base folder path
base_folder_path = "/home/dinesh/Documents/Projects/BTP-Face-Anti-Spoofing/Densenet-vit-logs/Protocol-logs/"

# Iterate through all folders in the base folder
for folder_name in os.listdir(base_folder_path):
    folder_path = os.path.join(base_folder_path, folder_name)

    # Ensure the path is a directory
    if os.path.isdir(folder_path):
        output_file = f"output/output_metrics_{folder_name}.txt"

        # Process log files and save metrics to output file
        process_log_files(folder_path, output_file)

        # Read metrics from the text file
        metrics_data = read_metrics_from_file(output_file)

        # Plot graphs using the metrics data
        plot_graphs_from_metrics(metrics_data, folder_name)
